# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [27]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

# Part 2

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [28]:
sms_spam_df = spark.read.csv('sms_spam.csv', header=True, inferSchema=True)
sms_spam_df.take(10)

[Row(type='ham', text='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'),
 Row(type='ham', text='Ok lar... Joking wif u oni...'),
 Row(type='spam', text="Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"),
 Row(type='ham', text='U dun say so early hor... U c already then say...'),
 Row(type='ham', text="Nah I don't think he goes to usf, he lives around here though"),
 Row(type='spam', text="FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"),
 Row(type='ham', text='Even my brother is not like to speak with me. They treat me like aids patent.'),
 Row(type='ham', text="As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callert

# Question 2.1

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`

In [29]:
# create sms_spam2_df below
sms_spam2_df = sms_spam_df.select(fn.when(fn.col('type')=='spam',1).otherwise(0).alias('type'),('text'))
#raise NotImplementedError()

In [30]:
# check here
sms_spam2_df.show()

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
|   1|FreeMsg Hey there...|
|   0|Even my brother i...|
|   0|As per your reque...|
|   1|WINNER!! As a val...|
|   1|Had your mobile 1...|
|   0|I'm gonna be home...|
|   1|SIX chances to wi...|
|   1|URGENT! You have ...|
|   0|I've been searchi...|
|   0|I HAVE A DATE ON ...|
|   1|XXXMobileMovieClu...|
|   0|Oh k...i'm watchi...|
|   0|Eh u remember how...|
|   0|Fine if that's th...|
|   1|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [31]:
# (5 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747]
)

# Question 2.2: tfidf feature engineering
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [32]:
# create a Pipeline transformer and name it 
from pyspark.ml.feature import Tokenizer
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
tokenizer = Tokenizer().setInputCol('text').setOutputCol('words')
count_vectorizer_estimator = CountVectorizer().setInputCol('words').setOutputCol('tf')
idf = IDF().setInputCol('tf').setOutputCol('tfidf')
tfidf_pipeline = Pipeline(stages=[tokenizer, count_vectorizer_estimator,idf]).fit(sms_spam2_df)
tfidf_pipeline.transform(sms_spam2_df)
#raise NotImplementedError()

DataFrame[type: int, text: string, words: array<string>, tf: vector, tfidf: vector]

In [33]:
tfidf_pipeline.transform(sms_spam2_df).show()

+----+--------------------+--------------------+--------------------+--------------------+
|type|                text|               words|                  tf|               tfidf|
+----+--------------------+--------------------+--------------------+--------------------+
|   0|Go until jurong p...|[go, until, juron...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|
|   0|Ok lar... Joking ...|[ok, lar..., joki...|(13525,[5,74,404,...|(13525,[5,74,404,...|
|   1|Free entry in 2 a...|[free, entry, in,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|
|   0|U dun say so earl...|[u, dun, say, so,...|(13525,[5,22,60,1...|(13525,[5,22,60,1...|
|   0|Nah I don't think...|[nah, i, don't, t...|(13525,[0,1,66,86...|(13525,[0,1,66,86...|
|   1|FreeMsg Hey there...|[freemsg, hey, th...|(13525,[0,2,6,10,...|(13525,[0,2,6,10,...|
|   0|Even my brother i...|[even, my, brothe...|(13525,[0,7,9,13,...|(13525,[0,7,9,13,...|
|   0|As per your reque...|[as, per, your, r...|(13525,[0,10,11,4...|(13525,[0,10,11,4...|

In [34]:
# (5 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

# Question 2.3: uppercase feature

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise. You can extract sequence of 3 or more uppercase letters by using the regular expression `[A-Z]{3,}`. You will use the function `fn.regexp_extract` to find those sequences and extract the first one (e.g., with index 0) and then use `fn.length` to compute the length of such sequence.

In [35]:
# create sms_spam3_df below
#sms_spam2_df.select('*', fn.length())
sms_spam3_df = sms_spam2_df.select(fn.when(fn.length(fn.regexp_extract(fn.col('text'),'[A-Z]{3,}',0))>2,1).otherwise(0).alias('has_uppercase'), ('type'),('text'))
sms_spam3_df.show()
#raise NotImplementedError()

+-------------+----+--------------------+
|has_uppercase|type|                text|
+-------------+----+--------------------+
|            0|   0|Go until jurong p...|
|            0|   0|Ok lar... Joking ...|
|            0|   1|Free entry in 2 a...|
|            0|   0|U dun say so earl...|
|            0|   0|Nah I don't think...|
|            0|   1|FreeMsg Hey there...|
|            0|   0|Even my brother i...|
|            0|   0|As per your reque...|
|            1|   1|WINNER!! As a val...|
|            1|   1|Had your mobile 1...|
|            0|   0|I'm gonna be home...|
|            1|   1|SIX chances to wi...|
|            1|   1|URGENT! You have ...|
|            0|   0|I've been searchi...|
|            1|   0|I HAVE A DATE ON ...|
|            1|   1|XXXMobileMovieClu...|
|            0|   0|Oh k...i'm watchi...|
|            0|   0|Eh u remember how...|
|            0|   0|Fine if that's th...|
|            1|   1|England v Macedon...|
+-------------+----+--------------

The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [36]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

[Row(has_uppercase=1, type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'),
 Row(has_uppercase=1, type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030'),
 Row(has_uppercase=1, type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info')]

In [37]:
# (5 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 2.4: Compare models

Using the following splits:

In [12]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [39]:
[training_df.count(), validation_df.count(), testing_df.count()]

[3311, 1709, 554]

**(5 pts)** Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$). Fit those pipelines to the appropriate data split.

1. Logistic regression with $\lambda=0$ and $\alpha=0$ (assign the fitted pipeline to `lr_pipeline1`)
2. Logistic regression with $\lambda=0.02$ and $\alpha=0.2$ (assign the fitted pipeline to `lr_pipeline2`)
3. Logistic regression with $\lambda=0.1$ and $\alpha=0.4$ (assign the fitted pipeline to `lr_pipeline3`)

In [14]:
# create lr_pipeline1, lr_pipeline2, and lr_pipeline3
from pyspark.ml.classification import LogisticRegression
lambda_par = 0.0
alpha_par = 0.0
en_lr1 = LogisticRegression().\
        setLabelCol('type').\
        setFeaturesCol('tfidf').\
        setRegParam(lambda_par).\
        setMaxIter(100).\
        setElasticNetParam(alpha_par)
lr_pipeline1 = Pipeline(stages=[tfidf_pipeline, en_lr1]).fit(training_df)

lambda_par = 0.02
alpha_par = 0.2
en_lr2 = LogisticRegression().\
        setLabelCol('type').\
        setFeaturesCol('tfidf').\
        setRegParam(lambda_par).\
        setMaxIter(100).\
        setElasticNetParam(alpha_par)
lr_pipeline2 = Pipeline(stages=[tfidf_pipeline, en_lr2]).fit(training_df)

lambda_par = 0.1
alpha_par = 0.4
en_lr3 = LogisticRegression().\
        setLabelCol('type').\
        setFeaturesCol('tfidf').\
        setRegParam(lambda_par).\
        setMaxIter(100).\
        setElasticNetParam(alpha_par)
lr_pipeline3 = Pipeline(stages=[tfidf_pipeline, en_lr3]).fit(training_df)
#raise NotImplementedError()

In [15]:
# (10 pts)
np.testing.assert_equal(type(lr_pipeline1), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline2), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline3), pipeline.PipelineModel)
np.testing.assert_array_equal([type(s) for s in lr_pipeline1.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline2.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline3.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

**(5 pts)** Use the evaluator object defined below to compute the area under the curve of your predictors. For example, to compute the area under the curve of pipeline 1 for a dataframe `df`, you would run

```python
evaluator.evaluate(lr_pipeline1.transform(df))
```

Assign the AUC of the three models to the variables `AUC1`, `AUC2`, and `AUC3`, and and assign the pipeline with the best model to a variable `best_model`

In [16]:
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='type')

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [18]:
# print the AUC for the three models as follows
# print("Model 1 AUC: ", evaluator.evaluate(....))
# etc
# finally, based on these, assign the best validated 
# model to a variable best_model
AUC1 = evaluator.evaluate(lr_pipeline1.transform(validation_df))
AUC2 = evaluator.evaluate(lr_pipeline2.transform(validation_df))
AUC3 = evaluator.evaluate(lr_pipeline3.transform(validation_df))
print("Model 1 AUC: ", AUC1)
print("Model 2 AUC: ", AUC2)
print("Model 3 AUC: ", AUC3)
best_model = lr_pipeline2
#raise NotImplementedError()

Model 1 AUC:  0.9606171229900157
Model 2 AUC:  0.9900130378096487
Model 3 AUC:  0.9543473851948426


In [19]:
# (5 pts)
np.testing.assert_array_equal([type(AUC1), type(AUC2), type(AUC3)],
                             [float, float, float])
# AUC less than 1
np.testing.assert_array_less([AUC1, AUC2, AUC3], [1, 1, 1])
# AUC more than 0.5
np.testing.assert_array_less([.5, .5, .5],
                            [AUC1, AUC2, AUC3])

# Question 2.5: Choose best model

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [40]:
# assign to AUC_best the AUC of the best model selected before
AUC_best= evaluator.evaluate(lr_pipeline2.transform(testing_df))
#raise NotImplementedError()

In [41]:
# (5 pts)
np.testing.assert_approx_equal(AUC_best, 
                               0.976126746201693, significant=2)

Using the same split and the best model, compute `precision`, `recall` and `f1_score`. You should first count the numbers in the confusion matrix, and then compute these metrics based on the formula.

In [77]:
# YOUR CODE HERE
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics
predictions = lr_pipeline2.transform(testing_df)
preds_and_labels = predictions.select(['prediction','type']).withColumn('type', fn.col('type').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','type'])

confusion_matrix = MulticlassMetrics(preds_and_labels.rdd.map(tuple)).confusionMatrix().toArray()
confusion_matrix
precision = confusion_matrix[1,1]/np.add(confusion_matrix[0,1], confusion_matrix[1,1])
recall = confusion_matrix[1,1]/np.add(confusion_matrix[1,0], confusion_matrix[1,1])
f1_score = 2*(precision*recall)/(precision+recall)

#raise NotImplementedError()

0.888888888888889

In [78]:
# (5 pts)
np.testing.assert_array_almost_equal([precision, recall, f1_score],
    [1.0, 0.7976190476190477, 0.8874172185430463],
                                     decimal=2)

# Question 2.6: Inference

Use the pipeline 2 fitted above (`lr_pipeline2`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: follow the `sentiment_analysis.ipynb` notebook in the repo**

In [24]:
# create positive_words and negative_words pandas dataframe below
import pandas as pd
vocabulary = tfidf_pipeline.stages[1].vocabulary
weights = lr_pipeline2.stages[-1].coefficients.toArray()
coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': weights})
positive_words=coeffs_df.sort_values('weight',ascending=False).head(20)
negative_words=coeffs_df.sort_values('weight',ascending=True).head(20)
#raise NotImplementedError()

In [25]:
# examine positive vocabulary
positive_words.head()

,word,weight
7263,sexy?,0.642738
3555,widelive.com/index.,0.588182
15,call,0.537161
12237,08714712388,0.504090
81,txt,0.495005


In [87]:
# examine solutions
negative_words.head()

,word,weight
1,i,-0.183463
3332,lose.,-0.074937
3371,fightng,-0.074937
3221,dificult,-0.074937
13,me,-0.065904


The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.590870</td>
    </tr>
    <tr>
      <th>12237</th>
      <td>08714712388</td>
      <td>0.533567</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.517100</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.513278</td>
    </tr>
    <tr>
      <th>9064</th>
      <td>gbp/sms</td>
      <td>0.468274</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.162493</td>
    </tr>
    <tr>
      <th>2444</th>
      <td>fighting</td>
      <td>-0.060939</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.059061</td>
    </tr>
  </tbody>
</table>

In [26]:
# (5 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.3701485692317927, significant=2)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.6661952507442954, significant=2)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

AssertionError: 
Items are not equal to 2 significant digits:
 ACTUAL: 8.675741267346242
 DESIRED: 8.370148569231793

# Question 2.7
Use the dataframe `sms_spam3_df` to create a model where the first feature is `has_uppercase` and the next set of features are the tfidf of the text. Perform feature engineering in all features using a max absolute scaler ([`MaxAbsScaler`](https://spark.apache.org/docs/2.0.2/ml-features.html#maxabsscaler)). Do a logistic regression on the resulting scaled features with regularization parameter $\lambda = 0.2$ and elastic net mixture $\alpha=0.1$ for the entire data (all of `sms_spam3_df`). Since you have scaled all features to be within the same range, you can compare them. 

**(5 pts)** with code and comments, answer below

1. is `has_uppercase` a feature that is positively or negative related to an SMS being spam?
2. what is the ratio of the coefficient of `has_uppercase` to the biggest positive tfidf coefficient?

In [80]:
# your code and comments below
tokenizer2 = Tokenizer().setInputCol('text').setOutputCol('words')
count_vectorizer_estimator2 = CountVectorizer().setInputCol('words').setOutputCol('tf')
idf2 = IDF().setInputCol('tf').setOutputCol('tfidf')
va = feature.VectorAssembler(inputCols=['has_uppercase', 'tfidf'], outputCol='features')
scaler = feature.MaxAbsScaler().setInputCol('features').setOutputCol('scaledFeatures')
tfidf_pipeline2 = Pipeline(stages=[tokenizer2, count_vectorizer_estimator2,idf2,va,scaler]).fit(sms_spam3_df)
tfidf_pipeline.transform(sms_spam3_df) 
lambda_par4 = 0.2
alpha_par4 = 0.1
en_lr4 = LogisticRegression().\
        setLabelCol('type').\
        setFeaturesCol('scaledFeatures').\
        setRegParam(lambda_par4).\
        setMaxIter(100).\
        setElasticNetParam(alpha_par4)
lr_pipeline4 = Pipeline(stages=[tfidf_pipeline2, en_lr4]).fit(sms_spam3_df)
#raise NotImplementedError()

In [86]:
lr_pipeline4.stages[1].coefficients[0]
#Since the coefficient for has_uppercase, which is the first element in the list of coefficients is 0.9289, has_uppercase is 
#positively related to an SMS being spam

0.9289178747599828

In [88]:
lr_pipeline4.stages[1].coefficients[0]/max(lr_pipeline4.stages[1].coefficients)
#The ratio of the coefficient of has_uppercase to the biggest positive tfidf coefficient is 0.4617

0.4617005798337623